In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense, Flatten
from sklearn.model_selection import train_test_split

In [2]:
# 1. 데이터 로드 및 전처리
# rms_x, rms_y, ..., vel_peak2peak_z 38개 피처
# 타임스텝마다 데이터가 있다고 가정

df = pd.read_csv('55285839-9b78-48d8-9f4e-573190ace016_data.csv') # 데이터 로드
X = df.iloc[:, 4:42].values  # 피처만 추출
y = df.iloc[:, 43:44].values # imbalance
# y = np.where(np.isnan(y), 0, y) # nan을 0(비정상)으로
y = np.where(y != 1, 0, 1) 

# 피처 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [3]:
# 2. 훈련 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [4]:
# 3. 1D-CNN 모델 구축
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # 이진 분류

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
# 모델 훈련
model.fit(X_train, y_train, epochs=10, batch_size=12, validation_data=(X_test, y_test))

Epoch 1/10
190/190 [==============================] - 1s 2ms/step - loss: 0.4904 - accuracy: 0.8151 - val_loss: 0.5096 - val_accuracy: 0.8063
Epoch 2/10
190/190 [==============================] - 0s 1ms/step - loss: 0.4666 - accuracy: 0.8160 - val_loss: 0.5143 - val_accuracy: 0.8063
Epoch 3/10
190/190 [==============================] - 0s 1ms/step - loss: 0.4665 - accuracy: 0.8169 - val_loss: 0.5132 - val_accuracy: 0.8063
Epoch 4/10
190/190 [==============================] - 0s 1ms/step - loss: 0.4606 - accuracy: 0.8160 - val_loss: 0.5097 - val_accuracy: 0.8028
Epoch 5/10
190/190 [==============================] - 0s 1ms/step - loss: 0.4579 - accuracy: 0.8173 - val_loss: 0.5112 - val_accuracy: 0.8063
Epoch 6/10
190/190 [==============================] - 0s 1ms/step - loss: 0.4548 - accuracy: 0.8160 - val_loss: 0.5031 - val_accuracy: 0.8063
Epoch 7/10
190/190 [==============================] - 0s 1ms/step - loss: 0.4539 - accuracy: 0.8156 - val_loss: 0.4972 - val_accuracy: 0.8063
Epoch 

In [6]:
# 훈련 과정의 손실 및 정확도 출력
print("\n훈련 과정의 최종 손실 및 정확도:")
final_loss, final_accuracy = model.evaluate(X_test, y_test)
print(f"테스트 손실: {final_loss:.4f}, 테스트 정확도: {final_accuracy:.4f}")


훈련 과정의 최종 손실 및 정확도:
18/18 [==============================] - 0s 511us/step - loss: 0.5069 - accuracy: 0.8011
테스트 손실: 0.5069, 테스트 정확도: 0.8011


In [17]:
from sklearn.metrics import classification_report

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.75).astype(int)  # 0.8 이상의 확률을 정상(1)으로 변환

# 정확도, 정밀도, 재현율, F1 스코어 계산 및 출력
print("\n분류 성능 보고서:")
print(classification_report(y_test, y_pred_classes, target_names=["비정상", "정상"]))

18/18 [==============================] - 0s 965us/step

분류 성능 보고서:
              precision    recall  f1-score   support

         비정상       0.29      0.35      0.31       110
          정상       0.83      0.79      0.81       458

    accuracy                           0.71       568
   macro avg       0.56      0.57      0.56       568
weighted avg       0.73      0.71      0.72       568

